# Week 4-5 solutions

## Question 1: Simulation for track reconstruction with silicon-detectors

### Learning objectives
In this question you will:

- learn about making more realistic simple monte carlo simulations
- learn how to emulate intrinsic resolution of multi-channel readout detectors
- learn the basics of calculating the kinematic parameters of a charged particles reconstructed in a silicon-based tracker
- learn how to calculate the radiation of materials and to take into account multiple scattering contributions


Silicon microstrip and pixel detectors play a prominent role in many particle physics experiments. Their good spatial resolution, ability to distinguish nearby particles and radiation hardness make them attractive options. You have studied the position resolution of such detectors in a previous homework. Today, we will learn about how they are used to reconstruct particle trajectories.

We'll consider a detector composed of 5 planes (_layers_) of slicon pixel detectors (see e.g. [LHCb VELO detector](http://lhcb-public.web.cern.ch/en/detector/VELO-en.html) or [LDMX](https://confluence.slac.stanford.edu/display/MME/Light+Dark+Matter+Experiment?preview=/210534721/228306759/LDMX_summary.pdf) detector). For simplicity we'll treat the problem as 2D problem. The silicon detectors will measure one coordinate (r) and the other one (z) is given by their known initial position (assumed to be corresponding to the center of the silicon sensor). The nominal origin of particles sits at $(r,z)=(0,0)$; each layer is parallel to the $r$ axis; the first layer sits at a distance $r=D=5$ cm from the nominal origin and each subsequent layer is placed at a distance $L=3$ cm from the previous one. A schematic view is shown below and further details are given in the questions below. No magnetic field is present (a magnetic field parallel to the $z$ or $r$ direction won't change significantly the problems below, this is the case for e.g. the detectors mentioned above).

![detector](detector.png)

### 1a.  (20 points)

Each layer can be thought as 1-D array of pixels with a width (along r) of $p=25\mu$m and a thickness of $t=150\mu m$ (see figure below). We assume that the pixels are placed such that $r=0$ corresponds to the center (in $r$ coordinate) of one of the pixels. Depending on the angle $\theta$ of the track, we expect a signal in one or more than one pixels (see figure below). Let's assume that there's no diffusion and the detector is not in any magnetic field; in this situation the electrons will drift along the electric field ($E$ in the figure, along the $z$ axis) inside the sensor straight to the electrode (situated at the top in the figure below) and the average amount of charge (i.e. energy released by ionization) recorded by each pixel will be proportional to the path length of the track in each pixel.

The goal of this question is two-folds:
- (i) to create a procedure/method (estimator) to estimate the position of intersection of the particle to the center (in $r$) of any given layer; the input of this estimator is only what signal(s) are recorded in the detector (that can be thought of as a reference position for the pixel that gave that signal).
- (ii) to determine the expected position resolution of this estimator as a function of the angle $\theta$ of the charged particle in the range $\theta < 30^\circ$ (was: $60^\circ$, revised to $30^\circ$ to simplify the problem). 

The resolution, in this context, refers to the square-root of the variance of the estimator of the particle's position at (the center of) the sensor. If we call $\sigma(r \vert \theta)_i$ the resolution in $r$ coordinate for the $i$-th layer given a true $\theta$ angle, it is common in this context to use in practice:
$$
\sigma(r \vert \theta)_i = \sqrt{Var[(r_{\textrm{meas}}(\textrm{pixels}(\theta)) - r_{\textrm{true}}(\theta)]}
$$
where $r_{\textrm{true}(\theta)}$ corresponds to the (exactly) calculated intersection of the charged particle with angle $\theta$ to the center (in $r$) of the layer, and $r_{\textrm{meas}}(\textrm{pixels}(\theta))$ corresponds to your method's estimate of the most likely position, given the signal(s) recorded in the detector (that implicitly depend on the theta angle of the charged particle). 

For this question, we'll actually assume each pixel can only record a 1 or 0 depending if the particle did or did not pass through any part of the pixel and produced enough ionization energy. You can assume that the particle needs to go through at least $37.5\mu$m of silicon before leaving a signal above the minimum detectable threshold. To further simplify, you can just focus on the first layer. We will use this resolution as representative to all layers later on.

While this can be computed analytically, I suggest using a Monte Carlo method. See the Hint below.

Interesting-to-know: the 1 or 0 readout is actually a relatively common arrangement when there's a need to minimize the amount of data transferred out of the detector.

#### <font color='red'>Hint:</font>

First we need to define an estimator for the position. It is easier to think about this problem dividing the cases of how many pixels register a signal.

As an example, let's consider the case we record a signal on just one pixel. We will define our estimator as the position of the center of that pixel in $r$ and and, focusing on the first layer only. The $z$ position is assumed to be known with good accuracy and to be $z=D$ in this case.

Focusing on small $\theta$ angles first, charged particles will hit only the central (in r) pixel if their intersection is in the range $|r| < p/2$. A small correction is that if the particle hits the next pixel but only travels less than $37.5~\mu$m in it. We'll neglect it here and it's much easier to take it into account using a simulation as outlined below (but can also be done analytically of course). This range corresponds to $\vert\theta\vert < \tan^{-1}(p/2D) = 2.5\cdot 10^{-6}$. 

The distribution of charged particle is assumed uniform in $\theta$. It is however easier to think about the distribution of $r_\textrm{true}$, the coordinate of the interseaction of the particle with the middle of the sensor. In this case the distribution of $r_\textrm{true}$ follows the distribution of $sin(\theta)$ which, locally, can be approximated by $\sin(\theta) \approx \theta$. Since the distribution is uniform in $\theta$ we can assume it to be also uniform in $r_\textrm{true}$. 

The expected resolution is therefore the one of a uniform distribution in this range, or - in formulas:
$$
\sigma^2(r\vert \theta) = \int_{-\infty}^{+infty} (r_e - r_\textrm{true})f(r_\textrm{true})dr_\textrm{true} = \frac{1}{p}\int_{-p/2}^{p/2} r_\textrm{true}^2 dr_\textrm{true} = p^2 / 12
$$
where $f(r_\textrm{true})$ is the pdf for $r_\textrm{true}$, that equals to $1/p$ for $-p/2 < r_\textrm{true} < p/2$ and $0$ elsewhere. We also used the fact that our estimator for this pixel is $r_e = 0$ in this range.

The same is approximately true for each case where we record a signal in only one pixel actually, since locally these approximations will also be valid around any point, given that $p << D$.

Now, how you would define an estimator in the case two pixels record a signal? 

In implementing this as a function, it is easier to split the problem in a few functions and use a Monte Carlo method to generate data and estimate the resolution from the standard deviation of our estimator applied to that data directly. 

An example of the possible structure is given in the beginning of the "Solutions" cells below, but feel free to use other approaches as well.

![pixels](pixels.png)

#### <font color='red'>Solution:</font>

First things first, let's define some numbers.

In [5]:
import math

pixel_p = 0.025 #mm
pixel_t = 0.150 #mm

layer_D = 50 #mm
layer_L = 30 #mm

We start by defining a function that will tell where the charged particle intersects the middle, lower, and upper point of our pixel sensor in $r$.

In [7]:
def getIntersections_r(theta, layer=1):
    r_middle = 0# replace with correct expression
    r_bottom = 0# replace with correct expression
    r_top = 0# replace with correct expression
    
    return r_middle, r_bottom, r_top

We then need our estimator. The estimator needs the position of (assumed contigous) pixels that registered a signal.

In [13]:
def calculatePosition_r(pixel_positions):
    #e.g. pixel_position = [0.0, 0.025]
    r = 0 # replace with your best estimate of the position
    
    return r

Now we need to create our Monte Carlo (simulated theta) and plot the residuals (estimated minus true value), separating the cases depending on how many pixels we hit

In [15]:
#Remember to set a seed (depending on what you're using), e.g.
# ROOT.gRandom.SetSeed(12345)

NSamples = 10000

def generate_particle():
    theta = 0.0
    #throw random number according to expected theta distribution (uniform), assign to theta
    #....
    return theta

for i in range(NSamples):
    #generate particle
    theta = generate_particle()
    
    #get the true value of intersections, and calculate what pixels fire
    r_c, r_l, r_h = getIntersections_r(theta, 1) #focus on layer 1 only
    length_in_pixels = 0 #replace with your calculation
    pixel_positions = []
    
    #here: fill pixel_positions with nominal (center in r) positions of fired pixels
    
    #now g
    r_m = calculatePosition_r(pixel_positions)
    
    #here: now make the plot of (r_m - r_c), either all together or, better, 
    #      making a separate plot depending on how many pixels fired
    
#here: now calculate the stddev of each plot to get the resolution

### 1b.  (20 points) [OPTIONAL]

In reality, our particles will not always leave the same ionization energy. As we've studied, the amount of energy by released by ionization follows a Landau distribution with average given by the Bethe-Block formula. 

In this question we will slightly simplify this description and instead use a Poisson distribution to describe the pdf of the energy deposit (most notably it significantly underestimates the tails at high-energy loss, when compared to a Landau distribution). We also will assume all particles considered are minimum-ionizing particle (m.i.p.) and in this case we expect the average number of electrons produced by ionization to be about 80 per $\mu$m. We can describe the number of electrons produced also using a Poisson distribution. Therefore you can use this number as the average of the Poisson distribution. 

Let's finally also assume that the minimum number of electrons we can detect is 3000 e^-. 

Create a function that computes the expected resolution given the angle $\theta$ of the charged particle. Compare to the resolution calculated in the question 1a.

#### <font color='red'>Solution:</font>

In [19]:
# return the resolution, using the results of the monte carlo simulation of question 1a or
# making a similar simulation for the more realistic case described in 1b
def meas_sigma(theta, layer=1):
    #compute the number of expected pixels
    
    #look-up what resolution you expect in that case
    
    #finish the function returning the expected resolution calculated in this function
    return resolution

### 1c. (20 points)

In absence of magnetic fields and neglecting for now multiple-scattering effects, the charged particle trajectory will follow a straight line: 

$$
r = r_0 + m\cdot z
$$

We now want to start making a small simulation program that will allow us to estimate - ultimately - the resolution on the estimated parameter $r_0$ of the reconstructed trajectory, under various conditions.

To start, create a function that takes as input the angle $\theta$ and, assuming the particle originates at $(r,z)=(0,0)$, calculate the position of the intersection points at each layer (in the middle of the silicon sensor). As a suggestion, keep the distance D as a parameter since later on we'll want to change it.

We then use this position to emulate a real "measurement". To do this, we simply add an offset in the $r$ direction, that is a random number that follows a Gaussian distribution with mean 0 and $\sigma$ equal to the expected resolution - calculated using the function you wrote in 1b - for the given input angle $\theta$:

```
measurements[i] = measurements[i] + ROOT.Math.Gaus(0, meas_sigma(theta))
```

We will assume that $z$ is known with much better accuracy and we know exactly its value.

The function should return two lists. The first one will be the list of the measurements (r,z) of the intersections (remember, $r$ will have the random offset added to it), while the second list will correspond to the expected resolution for the corresponding intersection of the first list (only consider uncertainty in the $r$ direction), calculated with the function you wrote in 1b. You can also decide to just return three lists (one for $r$ coordinates, one for $z$ coordinates, one for $\sigma(r)$ values), if this makes it easier for you later on.

#### <font color='red'>Solution:</font>

In [2]:
def get_measurements(theta, D=5.0):
    #write your code here
    
    # e.g. of measurements = [ [r_0, z_0], [r_1, z_1], ...] and uncertainties = [ sigma_r_0, sigma_r_1, ...]
    return measurements, uncertainties

### 1d. 

We now move focus to consider all measurements on the different planes together to constrain the particle's trajectory (track reconstruction).

Track reconstruction can be roughly divided in two main steps. First, a pattern recognition algorithm is used to associate a set of measurements with a single trajectory. Then, these measurements are fit to determine track parameters according to a model of the trajectory. This problem focuses on this last step, where track parameters are determined and for simplicity will only involve a single charged particle.

The aim of this question is to evaluate how the so-called intrinsic resolution on the pixel detector position measurements translates to an uncertainty on the estimated trajectory parameter $r_0$. 

The resolution on $r_0$ can be crucial for these experiments, since it allows to determine how likely is that the particle was really coming from the expected interaction point and not. The latter would be the case, for instance, if the particle is coming from the decay of a "mother" particle that travelled a little distance before decaying.

To achieve this goal, for a given track with angle $\theta$, we will:
- generate the measurements (using the function in 1c)
- fit the measurements and their uncertainties using the linear equation we expect $r = r_0 + m\cdot z$.
- retrieve the fitted value of $r_0$ and its uncertainty.

Wihtout resetting the random seed (i.e. setting it only once at the beginning), repeat the above for at least 100 tracks with $\theta = 30^\circ$ (in each of them the random numbers will be different and therefore the positions will differ) and plot the distribution of $r_0$ (note that what you want to plot is reality is $r_0 - r_0^{true}$, but $r_0^{true} = 0 $ in this case) and its significance $r_0 / \sigma(r_0)$, where $\sigma(r_0)$ is the uncertainty on the estimated parameter $r_0$ returned by the fit. Fit both distributions with a Gaussian and print the value of the $\sigma$ of these Gaussians. The former corresponds to the expected uncertainty on $r_0$, the latter is called a _pull_ plot and its \sigma should come out compatible with 1.0, if the expected uncertainty are correctly propagated (can you see why?).

Optionally, repeat the same excercise for various theta values and plot the expected $r_0$ resolution ($\sigma$ of the fitted Gaussian to the $r_0$ distribution) as a function of $\theta$.



#### <font color='red'>Solution:</font>

### 1e.  (20 points)

Each silicon detector layer is actually made of a carbon support structure, silicon, and an epoxy glue ($C_{18}H_{18}O_{3}$, density $\rho=1 g/cm^3$). We'll neglect everything else. Each of the three components is weighed before assembly and their relative mass is respectively 50%, 40% and 10% of the total. Estimate the radiation length ($X_0$) corresponding to each layer. Estimate as well the thickness of each layer.

The radiation lengths of a compound can be computed similarly to the case of mass stopping power dE/dx, where a weighted average of $1/X_0$, with weight given by the relative mass. This weighted average gives the resulting $1/X_0$ of the compound, i.e.
$$
\frac{1}{X_{0,\textrm{compound}}} = \sum_i \frac{w_i}{X_{0, i}}
$$
where $w_i$ are the mass fraction of each component.

Notes: the drawing below is _not_ in scale; the thickness of the silicon layer was already given above; the density of silicon and carbon can easily be looked up.

![Silicon Module Stackup](silicon-module-stackup.png)

#### <font color='red'>Solution:</font>

### 1f. (20 points)

We'll now revist the `get_measurements` function we wrote in 1c to also include effects from multiple scattering, in addition to intrinsic resolution.

The average scattering angle from multiple scattering can be estimated given the $X_0$ values you calculated above. To take this into account correctly, now you need to calculate the intersection with each layer and adjust the outgoing trajectory when moving (usually referred to as _extrapolating_) from one layer to the next one, as depicted below.

At each step, add a multiple scattering contribution which is modelled via a random number that follows a Gaussian distribution with mean 0 and $\sigma$ equal to the expected multiple scattering contribution.

To be able to use these new simulated measurements consistently in a fit, we also need to add to the expected resolution we return in this function the expected effect of multiple scattering from previous layers in quadrature. In other words your final expected uncertainty on the measurement $m$ should be $\sigma = \sqrt{\sigma_m^2 + \sigma_{MS}^2}$, where $\sigma_m$ is the intrinsic detector uncertainty you evaluated before and $\sigma_{MS}$ is the contribution of multiple scattering from _all_ planes before the one being considered (note: the one being considered does not contribute since it will only deviate the trajectory from its path for planes that come afterwards).

![det_ms](detector_ms.png)

#### <font color='red'>Solution:</font>

In [3]:
def get_measurements_ms(theta, D=5.0):
    
    return measurements, uncertainties

### 1g.  (20 points)

Now repeat the plots of step 1d, but using instead the measurements returned by the `get_measurements_ms` function, that include the multiple scattering contribution. Compare the new resolution to the one obtained before (when multiple scattering effects were ignored). Does the pull width come out compatible with 1? if not, can you think of why that would not be the case? 

The plot of resolution vs $\theta$ is still optional.

#### <font color='red'>Solution:</font>

### 1h.  (20 points)

As a final excercise, vary the distance D of the first layer (keeping the distance L between layers fixed). Show how the uncertainty on $z_0$ varies as a function of the distance D in the range $1-10$cm.

#### <font color='red'>Solution:</font>

### 1i. [OPTIONAL, only for the braves..] (a LOT of points)

Ok, I said the previous one was the last one... but if you're brave there's one more thing you can do.

The fitting method above is not ideal since it does ignore the fact that once we have a measurement on a given layer, this measurement is giving us information on how much multiple scattering has happened so far. 

To improve on that, instead of fitting your straight line with uncertainties that take into account the $MS$ contribution, you can define a custom $\chi^2$ (likelihood) function of the form:

$$
\chi^2 = \sum_k \frac{(r_{k, \textrm{meas.}} - r_{k, \textrm{exp}}(\delta\theta))^2}{\sigma_{k, \textrm{meas.}}}^2 + \sum_i \frac{\delta\theta_i^2}{\sigma(\delta\theta_i)^2}
$$

where the sum goes through the measurements in each layer, starting from the layer closer to the particle's origin and 
- $r_\textrm{meas}$ is the measured (simulated) position used as input to the fit;
- $r_\textrm{exp}$ is the expected position on each layer given the information of the _measured_ positions in the previous layer(s). I've indicated that as $\delta\theta = (\delta\theta_1, \delta\theta_2, ...)$ in the equation above, meaning $\delta\theta_1$ is the MS contribution from the first layer's material to the extrapolation to the second layer, etc.. and the only relevant ones for each $r_k$ determinations are the ones with index $< k$. There's multiple ways to do this in practice, feel free to choose what feels "right" in the way you're thinking about this problem; 
- $\sigma_\textrm{meas}$ now only accounts for the intrinsic uncertainty on the measurement poisition (neglecting MS from previous layers);
- the MS contributions (only from the previous layers!) are now taken into account with an extra "penalty" factor in the likelihood/$\chi^2$ that penalizes large scattering angle with their expected (Gaussian) pdf.   

Such an approach is much more similar to a realistic track-fitting approach used nowadays. It has the advantage that it uses the information of the measurements to constrain (fit) the multiple scattering contributions at the same time.

Calculate again the $r_0$ resolution and pull plot with this method and compare to what you've opbtained before. 

#### <font color='red'>Solution:</font>